### Tipos de Testes de Hipóteses

Testes de hipótese servem para verificar, se uma afirmação sobre uma população é plausível ou deve ser rejeitada. Eles auxiliam a tomada de decisões com base em dados.

- Testes de Normalidade
- Testes Paramétricos
    - Usados quando os dados seguem a **Distribuição Normal**.

- Testes Não Paramétricos
    - Usados quando os dados são **Ordinais** ou não seguem a **Distribuição Normal**
- Teste Bayesianos
- Testes de Regressão

In [1]:
""" Imports """

import pandas as pd
import numpy as np
import math

import plotly.express as px
import plotly.graph_objects as go

from scipy import stats
from scipy.stats import norm, shapiro, kstest, pearsonr, linregress

### Testes de Normalidade

In [2]:
""" Datasets randomico para ser usado nos testes de normalidade para Distribuição Normal. """

dados_distr_normal = np.random.normal(loc=0, scale=1, size=100)
dados_distr_normal

array([ 0.36474476,  0.72499995,  1.66431661,  0.26579308, -0.97332762,
       -0.18729841, -0.1262835 ,  0.54202977, -0.76783924,  1.17380143,
       -0.22335833, -0.13543737, -0.85500957, -0.23123644,  0.16532989,
       -1.60362654,  0.78469122, -0.46468576, -2.52429408, -1.43296226,
        0.8941813 , -1.2031716 , -0.17102027, -0.35436895,  0.70167272,
       -0.3933818 ,  1.03408513, -0.07507396,  1.1016812 , -0.21972704,
       -0.94099182, -0.33409686,  0.52687106,  0.80426014, -0.88554958,
       -0.14685384,  0.39268735, -0.13382319,  0.71789055, -1.53549392,
       -0.61917891, -1.329201  ,  0.92391458, -1.43709257, -1.12263779,
       -0.53535527,  1.90432015, -0.5375203 ,  1.48594908, -0.98124274,
       -0.74928218,  0.49990491, -0.06099089, -0.5339662 , -0.08374268,
       -1.89292479, -1.78792779, -0.62642138, -0.87582121, -1.54566502,
       -0.96039561, -0.32593844,  1.52199872,  0.21315044, -0.45280657,
        0.38943253,  2.68912536,  0.19003347, -1.60829296, -1.22

In [3]:
""" Gráfico da curva normal com base nos dados obtidos. """

def grafico_distribuicao(data):
    # Criar curva normal
    x = np.linspace(min(data), max(data), 200)
    y = norm.pdf(x, np.mean(data), np.std(data))
    media = np.mean(data)

    df = pd.DataFrame({"x": x, "densidade": y})

    # Criar gráfico
    fig = px.line(df, x="x", y="densidade", title="Curva de Distribuição")
    # Adicionar a linha da média
    fig.add_vline(x=media, line_dash="dash", line_color="red", annotation_text=f"Média = {media:.2f}")

    fig.show()

In [4]:
grafico_distribuicao(dados_distr_normal)

In [5]:
""" Datasets randomico para ser usado nos testes de normalidade para Distribuição Não Normal. """

dados_distr_nao_normal = np.random.exponential(scale=1, size=100)
dados_distr_nao_normal

array([1.56434882, 0.72614617, 0.41361247, 0.11878707, 0.15970005,
       0.78513906, 0.16661595, 1.43097159, 0.23330707, 0.74331237,
       0.2757726 , 0.58327737, 1.29174712, 1.89359085, 0.88462689,
       0.4775324 , 0.00740182, 0.14907102, 0.10938543, 0.2861975 ,
       3.15531331, 0.65729357, 0.54367285, 1.54961265, 0.26464039,
       0.49573978, 0.2945901 , 0.71705816, 0.2950028 , 0.46661456,
       1.1799342 , 0.61173627, 0.03545743, 1.42393555, 0.96497154,
       1.99401728, 0.05934981, 0.27180987, 4.05828574, 0.20642558,
       1.17032429, 0.52765138, 0.67361161, 2.05498438, 0.60792769,
       2.52959532, 0.16849803, 0.28837697, 0.87955788, 0.36387308,
       1.61975795, 1.23136074, 1.93902171, 1.1598925 , 0.69111203,
       0.411444  , 1.66504781, 0.47405575, 0.65252066, 1.26379959,
       1.03837448, 2.45626074, 1.56929781, 0.36031301, 3.62470202,
       0.9145026 , 0.17058047, 0.38072647, 5.07227325, 0.29834568,
       1.58680886, 0.71737966, 0.36901628, 0.81643603, 0.39123

In [6]:
grafico_distribuicao(dados_distr_nao_normal)

Os **Testes de Normalidade** servem para verificar se um conjunto de dados seguem uma **Distribuição Normal** (se os dados se concentram em torno da média)

Sendo os tipos de testes de normalidade:

- **Shapiro-Wilk**
- **Kolmogorov-Smirnov**
- Anderson-Darling
- Jaque-Bera

#### Shapiro Wilk

Serve para verificar se um conjunto de dados **segue uma distribuição normal**.  
É especialmente indicado para **amostras pequenas ou médias**.

**Fórmula:**
$$
W = \frac{\left(\sum_{i=1}^{n} a_i x_{(i)}\right)^2}
{\sum_{i=1}^{n} (x_i - \bar{x})^2}
$$

- x_(i): valores da amostra ordenados em ordem crescente  
- xᵢ: valores observados  
- x̄: média da amostra  
- aᵢ: constantes calculadas a partir da distribuição normal teórica  
- n: tamanho da amostra  

**Resultado:**
- **p ≥ 0,05** → os dados **podem seguir** uma distribuição normal  
- **p < 0,05** → os dados **provavelmente não seguem** uma distribuição normal

In [7]:
""" Função para teste de normalidade usando Shapiro Wilk """

def testar_normalidade_shapiro(data):
    stat, p = shapiro(data)

    print(f"Estatística W = {stat:.4f}") # Similaridade dos dados com uma normal, sendo 1 = perfeitamente normal
    print(f"p-valor = {p:.4f}")

    if p > 0.05:
        print("✔ A distribuição pode ser considerada normal (falha em rejeitar H0).")
    else:
        print("✘ A distribuição NÃO é normal (rejeita H0).")

In [8]:
testar_normalidade_shapiro(dados_distr_normal)

Estatística W = 0.9917
p-valor = 0.8024
✔ A distribuição pode ser considerada normal (falha em rejeitar H0).


In [9]:
testar_normalidade_shapiro(dados_distr_nao_normal)

Estatística W = 0.8233
p-valor = 0.0000
✘ A distribuição NÃO é normal (rejeita H0).


#### Kolmogorov-Smirnov

Compara a distribuição dos seus dados com **qualquer distribuição teórica**
(normal, exponencial, uniforme, etc.).

É **menos sensível ao tamanho da amostra** do que o teste de Shapiro–Wilk
quando usado para normalidade.

**Fórmula:**
$$
D = \sup_x \left| F_n(x) - F(x) \right|
$$

- $F_n(x)$: função de distribuição empírica da amostra  
- $F(x)$: função de distribuição acumulada da distribuição teórica escolhida  
- $\sup$: maior diferença absoluta observada  
- $D$: estatística do teste Kolmogorov–Smirnov  

**Interpretação do resultado**
- **p ≥ 0,05** → os dados **combinam** com a distribuição escolhida  
- **p < 0,05** → os dados **não seguem** a distribuição escolhida


In [10]:
""" Função para teste de normalidade usando Kolmogorov-Smirnov """

def testar_normalidade_ks(data):
    media = np.mean(data)
    desvio = np.std(data, ddof=1)

    # Teste KS comparando com uma normal teórica
    stat, p = kstest(data, 'norm', args=(media, desvio))

    print(f"Estatística KS = {stat:.4f}") # Maior diferença entre distribuições acumuladas, sendo 0 = distribuições identicas
    print(f"p-valor = {p:.4f}")

    if p > 0.05:
        print("✔ A distribuição pode ser considerada normal (falha em rejeitar H0).")
    else:
        print("✘ A distribuição NÃO é normal (rejeita H0).")

In [11]:
testar_normalidade_ks(dados_distr_normal)

Estatística KS = 0.0754
p-valor = 0.5943
✔ A distribuição pode ser considerada normal (falha em rejeitar H0).


In [12]:
testar_normalidade_ks(dados_distr_nao_normal)

Estatística KS = 0.1483
p-valor = 0.0220
✘ A distribuição NÃO é normal (rejeita H0).


### Testes Paramétricos

Para realizar um teste paramétrico, é necessário:
- Que os valores sigam uma distribuição normal.
- Sejam valores quantitativos (intervalar ou razão).

Sendo os tipos de testes paramétricos:
- Teste Z
- Teste t de Student
- Anova
- Correlação de Person
- Regressão Linear

---

#### Teste Z

O Teste Z é um teste estatístico paramétrico utilizado para avaliar se a média de uma amostra — ou uma proporção amostral — é estatisticamente diferente de um valor previamente conhecido da população.

Esse teste é indicado quando o desvio padrão populacional é conhecido e o tamanho da amostra é grande (geralmente n ≥ 30).

**Teste para as Médias**  
Utilizado quando se analisa o valor médio de uma variável numérica:
$$
Z = \frac{\bar{x} - \mu}{\sigma / \sqrt{n}}
$$

**Teste Z para as Proporções**  
Utilizado quando se analisa a fração de elementos que possuem uma determinada característica em relação ao total da amostra:
$$
Z = \frac{\hat{p} - p}{\sqrt{\frac{p(1 - p)}{n}}}
$$

- p < 0.05 → Rejeita-se a hipótese nula (H₀)
- p ≥ 0.05 → Não se rejeita a hipótese nula (H₀)

In [ ]:
def teste_z_media_plot(amostra, mu, sigma, alpha=0.05):

    amostra = np.array(amostra)
    n = len(amostra)
    media = amostra.mean()

    z = (media - mu) / (sigma / np.sqrt(n))
    p_valor = 2 * (1 - norm.cdf(abs(z)))

    # Gráfico
    x = np.linspace(-4, 4, 400)
    y = norm.pdf(x)
    z_critico = norm.ppf(1 - alpha / 2)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Normal'))

    fig.add_trace(go.Scatter(
        x=[z, z], y=[0, norm.pdf(z)],
        mode='lines', name=f'Z observado = {z:.2f}'
    ))

    fig.add_trace(go.Scatter(
        x=x[x >= z_critico], y=y[x >= z_critico],
        fill='tozeroy', opacity=0.3, name='Região crítica'
    ))

    fig.add_trace(go.Scatter(
        x=x[x <= -z_critico], y=y[x <= -z_critico],
        fill='tozeroy', opacity=0.3, showlegend=False
    ))

    fig.update_layout(
        title=f"Teste Z clássico (z={z:.2f}, p={p_valor:.4f})",
        xaxis_title="Z",
        yaxis_title="Densidade"
    )

    fig.show()
    decisao = "Rejeita H₀" if p_valor < alpha else "Não rejeita H₀"
    return z, p_valor, decisao

In [55]:
dados = [52, 50, 51, 53, 54, 49, 50, 52, 51, 53]

z, p, decisao = teste_z_media_plot(dados, mu=50, sigma=2)

print("Z:", z)
print("p-valor:", p)
print("Decisão:", decisao)

Z: 2.3717082451262845
p-valor: 0.017706065807366667
Decisão: Rejeita H₀


In [57]:
def teste_z_proporcao_plot(sucessos, n, p0, alpha=0.05):
    """
    sucessos : número de sucessos observados
    n        : tamanho da amostra
    p0       : proporção populacional (H0)
    alpha    : nível de significância
    """

    # Proporção amostral
    p_hat = sucessos / n

    # Estatística Z
    z = (p_hat - p0) / np.sqrt(p0 * (1 - p0) / n)

    # p-valor (bicaudal)
    p_valor = 2 * (1 - norm.cdf(abs(z)))

    # Curva normal
    x = np.linspace(-4, 4, 400)
    y = norm.pdf(x)

    # Valor crítico
    z_critico = norm.ppf(1 - alpha / 2)

    fig = go.Figure()

    # Distribuição normal
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        name='Distribuição Normal'
    ))

    # Linha do Z observado
    fig.add_trace(go.Scatter(
        x=[z, z],
        y=[0, norm.pdf(z)],
        mode='lines',
        name=f'Z observado = {z:.2f}'
    ))

    # Regiões críticas
    fig.add_trace(go.Scatter(
        x=x[x >= z_critico],
        y=y[x >= z_critico],
        fill='tozeroy',
        name='Região crítica',
        opacity=0.3
    ))

    fig.add_trace(go.Scatter(
        x=x[x <= -z_critico],
        y=y[x <= -z_critico],
        fill='tozeroy',
        showlegend=False,
        opacity=0.3
    ))

    fig.update_layout(
        title=f"Teste Z para Proporções (z={z:.2f}, p={p_valor:.4f})",
        xaxis_title="Z",
        yaxis_title="Densidade"
    )
    fig.show()
    decisao = "Rejeita H₀" if p_valor < alpha else "Não rejeita H₀"
    return z, p_valor, decisao

In [58]:
z, p, decisao = teste_z_proporcao_plot(sucessos=60, n=100, p0=0.5)

print("Z:", z)
print("p-valor:", p)
print("Decisão:", decisao)

Z: 1.9999999999999996
p-valor: 0.04550026389635842
Decisão: Rejeita H₀


#### Teste t de Student

O **Teste t de Student** é um teste estatístico utilizado para comparar **médias**, especialmente quando o tamanho da amostra é pequeno (geralmente **𝑛 < 30**) e o **desvio padrão** populacional é **desconhecido**.

Ele avalia se a diferença entre a média observada e a média hipotética é estatisticamente significativa ou se pode ser atribuída ao acaso.

Fórmula:
$$
t = \frac{\bar{x} - \mu}{s / \sqrt{n}}
$$

- p < 0.05 → Rejeita-se a hipótese nula (H₀)
- p ≥ 0.05 → Não se rejeita a hipótese nula (H₀)

In [ ]:
def teste_t_uma_amostra(amostra, mu=0, alpha=0.05):
    n = len(amostra)
    media = sum(amostra) / n
    variancia = sum((x - media) ** 2 for x in amostra) / (n - 1)
    s = math.sqrt(variancia)

    t = (media - mu) / (s / math.sqrt(n))
    p = 2 * (1 - stats.t.cdf(abs(t), df=n-1))

    decisao = "Rejeita H0" if p < alpha else "Não rejeita H0"

    return t, p, decisao

In [6]:
dados = [10, 12, 9, 11, 13, 10, 12]

t, p, decisao = teste_t_uma_amostra(dados, mu=11)

print(f"t = {t:.3f}")
print(f"p-valor = {p:.4f}")
print(decisao)

t = 0.000
p-valor = 1.0000
Não rejeita H0


#### Anova

O teste ANOVA (Analysis of Variance — Análise de Variância) é um teste estatístico utilizado para comparar as médias de três ou mais grupos simultaneamente.

Ele avalia se pelo menos uma das médias difere estatisticamente das demais, analisando a relação entre a variabilidade entre os grupos e a variabilidade dentro dos grupos.
Quando a variabilidade entre grupos é significativamente maior do que a variabilidade interna, conclui-se que nem todas as médias são iguais.

A estatística do teste é dada por:

$$
F = \frac{SS_{\text{entre}} / (k - 1)}{SS_{\text{dentro}} / (n - k)}
$$

Resultado:
- p < 0.05 → rejeita-se H₀  
Existe diferença estatisticamente significativa entre as médias

- p ≥ 0.05 → não se rejeita H₀  
Não há evidência suficiente de diferença entre as médias

In [ ]:
def anova_um_fator(grupos, alpha=0.05):
    """
    Realiza o teste ANOVA de um fator.

    Parâmetros:
    grupos : lista de listas
             Cada lista interna representa um grupo
    alpha  : nível de significância

    Retorna:
    F       : estatística F
    p       : p-valor
    decisao : decisão do teste
    """

    F, p = stats.f_oneway(*grupos)
    decisao = "Rejeita H0" if p < alpha else "Não rejeita H0"
    return F, p, decisao

In [5]:
grupo1 = [10, 12, 9, 11]
grupo2 = [14, 15, 13, 16]
grupo3 = [9, 8, 10, 9]

F, p, decisao = anova_um_fator([grupo1, grupo2, grupo3])

print(f"F = {F:.3f}")
print(f"p-valor = {p:.4f}")
print(decisao)

F = 24.250
p-valor = 0.0002
Rejeita H0


#### Correlação de Person

A **correlação de Pearson** é uma medida estatística que avalia o **grau (força)** e a **direção** da **relação linear** entre duas variáveis **quantitativas**.

O valor de **r** indica a **força** e a **direção** da relação
- **r > 0**: relação linear positiva
- **r < 0**: relação linear negativa
- **r = 0**: ausência de relação linear

O **p-value** é usado para verificar se a correlação observada é estatisticamente **significativa**:
- **p < 0,05** → correlação estatisticamente significativa
- **p ≥ 0,05** → correlação não significativa

Fórmula da correlação de Pearson:
$$
r = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}
{\sqrt{\sum (x_i - \bar{x})^2 \sum (y_i - \bar{y})^2}}
$$

Interpretação do resultado:
- $r > 0$ → relação linear positiva
- $r < 0$ → relação linear negativa
- $r \approx 0$ → ausência de relação linear

Teste de significância:
- p ≥ 0,05 → correlação não estatisticamente significativa
- p < 0,05 → correlação estatisticamente significativa

In [ ]:
def correlacao_pearson_plot(x, y):
    x = np.array(x)
    y = np.array(y)

    # Correlação de Pearson
    r, p_valor = pearsonr(x, y)

    # Regressão linear (y = ax + b)
    a, b = np.polyfit(x, y, 1)
    y_pred = a * x + b

    # Gráfico
    fig = go.Figure()

    # Pontos
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name='Dados'
    ))

    # Reta de regressão
    fig.add_trace(go.Scatter(
        x=x,
        y=y_pred,
        mode='lines',
        name=f'Reta (r={r:.2f})'
    ))

    fig.update_layout(
        title=f'Correlação de Pearson (r={r:.2f}, p={p_valor:.4f})',
        xaxis_title='X',
        yaxis_title='Y'
    )
    fig.show()

    if r > 0:
        print("Relação linear positiva")
    elif r < 0:
        print("Relação linear negativa")
    elif r == 0:
        print("Ausencia de relação linear")

    if p_valor >= 0.05:
        print("Coorrelação não significativa")
    else:
        print("Coorrelação significativa")

    return r, p_valor, a, b

In [34]:
x = [1, 2, 3, 4, 5]
y = [2, 4, 5, 8, 10]

r, p, a, b = correlacao_pearson_plot(x, y)

print("r:", r)
print("p-valor:", p)
print("reta: y =", a, "* x +", b)

Relação linear positiva
Coorrelação significativa
r: 0.9901475429766743
p-valor: 0.00117221644471691
reta: y = 2.0 * x + -0.19999999999999896


#### Regressão Linear

Regressão linear é um método estatístico usado para modelar e testar a relação entre uma variável dependente (Y) e uma ou mais variáveis independentes (X), assumindo que essa relação pode ser aproximada por uma linha reta.

Fórmula:
$$
Y = \beta_0 + \beta_1 X + \varepsilon
$$

- Y = variável dependente  
- X = variável independente  
- β₀ = intercepto  
- β₁ = coeficiente angular  
- ε = erro aleatório  

Forma estimada do modelo:
$$
\hat{Y} = b_0 + b_1 X
$$

Resultado:  
- H₀ (hipótese nula): o coeficiente da regressão = 0 (X não tem efeito sobre Y)  
- H₁ (hipótese alternativa): o coeficiente ≠ 0 (X tem efeito sobre Y)  

In [ ]:
def teste_hipotese_regressao_scipy(x, y, alpha=0.05):
    resultado = linregress(x, y)

    coef = resultado.slope
    intercepto = resultado.intercept
    p_valor = resultado.pvalue
    decisao = "Rejeitar H0" if p_valor < alpha else "Falha em rejeitar H0"

    # Criando gráfico Plotly
    y_pred = [intercepto + coef * xi for xi in x]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Dados'))
    fig.add_trace(go.Scatter(x=x, y=y_pred, mode='lines', name='Regressão', line=dict(color='red')))
    fig.update_layout(
        title=f"Regressão Linear: coef={coef:.3f}, p-valor={p_valor:.3f}",
        xaxis_title="X",
        yaxis_title="Y",
        template="plotly_white"
    )

    return {
        "coeficiente": coef,
        "intercepto": intercepto,
        "p_valor": p_valor,
        "decisao": decisao,
        "figura": fig
    }

In [3]:
# Exemplo de uso
x = [1, 2, 3, 4, 5]
y = [2, 4, 5, 4, 5]

resultado = teste_hipotese_regressao_scipy(x, y)
print(resultado["coeficiente"], resultado["p_valor"], resultado["decisao"])
resultado["figura"].show()

0.6000000000000001 0.12402706265755442 Falha em rejeitar H0
